# Data preprocessing

Notebook for preprocessing overtrading datasets.


## Concepts

- **Session**: one fixed-time window (e.g., each 15-minute window is a session).
- **Core window**: same as session in this setup.
- **Overtrading indicators**: per-window indicators appended to each row.

This notebook builds one **training row per window**.

**Note:** each row corresponds to a single window and includes both the core-window
features and the per-window overtrading indicators. We also include `balanced_example.csv`
from `trading_datasets` to provide calm baseline windows, and add `is_calm`
for **labeling only** (not as a model feature).


In [41]:
import pandas as pd
from pathlib import Path

# Load mock behaviors datasets
base_dir = Path("../../../")
mock_dir = base_dir / "mock_behaviors"
if not mock_dir.exists():
    # Backward-compat for existing folder name
    mock_dir = base_dir / "mock_behaviours"

# All training files live in mock_behaviours/ (*_example.csv)
files = sorted(mock_dir.glob("*_example.csv"))

files


[PosixPath('../../../mock_behaviours/balanced_example.csv'),
 PosixPath('../../../mock_behaviours/calm_1_example.csv'),
 PosixPath('../../../mock_behaviours/calm_2_example.csv'),
 PosixPath('../../../mock_behaviours/calm_3_example.csv'),
 PosixPath('../../../mock_behaviours/loss_averse_example.csv'),
 PosixPath('../../../mock_behaviours/overtrading_example.csv'),
 PosixPath('../../../mock_behaviours/revenge_example.csv')]

In [42]:
# Load one dataset
sample_path = files[0] if files else None
sample_path


PosixPath('../../../mock_behaviours/balanced_example.csv')

In [43]:
import math


def load_trades(csv_path: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
    df = df.sort_values("timestamp").reset_index(drop=True)
    df["notional"] = df["quantity"] * df["entry_price"]
    return df


df = load_trades(sample_path) if sample_path else pd.DataFrame()
df.head()


,timestamp,asset,side,quantity,entry_price,exit_price,profit_loss,balance,notional
0,2025-01-01 13:34:29+00:00,TSLA,buy,2.9176,220.0000,215.9315,-13.59,9986.41,641.872000
1,2025-01-01 13:38:09+00:00,TSLA,buy,2.4824,215.9315,213.6644,-7.06,9979.35,536.028356
2,2025-01-02 03:24:58+00:00,TSLA,buy,2.8905,213.6644,216.5682,6.74,9986.08,617.596948
3,2025-01-02 03:37:45+00:00,TSLA,buy,2.9411,216.5682,212.6641,-13.19,9972.89,636.948733
4,2025-01-03 06:00:12+00:00,TSLA,sell,2.5624,212.6641,213.3491,-3.22,9969.68,544.930490


In [44]:
def compute_core_window_vector(
    df: pd.DataFrame,
    window_minutes: int = 15,
    stride_minutes: int = 5,
    eps: float = 1e-9,
) -> pd.DataFrame:
    if df.empty:
        return pd.DataFrame()

    df = df.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
    df = df.sort_values("timestamp")

    start = df["timestamp"].min().floor("min")
    end = df["timestamp"].max().ceil("min")

    window_starts = pd.date_range(start=start, end=end, freq=f"{stride_minutes}min", tz="UTC")

    rows = []
    for ws in window_starts:
        we = ws + pd.Timedelta(minutes=window_minutes)
        w = df[(df["timestamp"] >= ws) & (df["timestamp"] < we)].copy()

        n_trades = len(w)
        trade_rate_per_min = n_trades / window_minutes

        if n_trades >= 2:
            w["dt_prev_sec"] = w["timestamp"].diff().dt.total_seconds()
            gaps = w["dt_prev_sec"].dropna()
            median_gap_sec = float(gaps.median())
            mean_gap_sec = float(gaps.mean())
            gap_cv = float(gaps.std(ddof=0) / (gaps.mean() + eps))
            burst_frac = float((gaps <= 60).mean())
        else:
            median_gap_sec = float("nan")
            mean_gap_sec = float("nan")
            gap_cv = float("nan")
            burst_frac = float("nan")

        n_assets = int(w["asset"].nunique()) if n_trades else 0
        if n_trades:
            top_asset_share = float(w["asset"].value_counts(normalize=True).iloc[0])
        else:
            top_asset_share = float("nan")

        if n_trades >= 2:
            asset_switch_rate = float((w["asset"].iloc[1:].values != w["asset"].iloc[:-1].values).mean())
        else:
            asset_switch_rate = float("nan")

        notional_sum = float(w["notional"].sum()) if n_trades else 0.0

        if n_trades and "balance" in w.columns:
            window_start_balance = float(w["balance"].iloc[0])
            turnover = notional_sum / (window_start_balance + eps)
        else:
            turnover = float("nan")

        pnl_sum = float(w["profit_loss"].sum()) if n_trades else 0.0
        pnl_mean = float(w["profit_loss"].mean()) if n_trades else float("nan")
        pnl_std = float(w["profit_loss"].std()) if n_trades else float("nan")

        if n_trades:
            wins = w[w["profit_loss"] > 0]
            losses = w[w["profit_loss"] < 0]
            win_rate = float((w["profit_loss"] > 0).mean())
            avg_gain = float(wins["profit_loss"].mean()) if len(wins) else float("nan")
            avg_loss_abs = float(losses["profit_loss"].abs().mean()) if len(losses) else float("nan")
            payoff_ratio = avg_gain / (avg_loss_abs + eps) if not math.isnan(avg_gain) else float("nan")
            p90 = float(w["profit_loss"].quantile(0.90))
            p10 = float(w["profit_loss"].quantile(0.10))
            p50 = float(w["profit_loss"].quantile(0.50))
            pnl_skew_proxy = (p90 + p10) / (abs(p50) + eps)
        else:
            win_rate = float("nan")
            payoff_ratio = float("nan")
            pnl_skew_proxy = float("nan")

        rows.append(
            {
                "window_start": ws,
                "window_end": we,
                "n_trades": n_trades,
                "trade_rate_per_min": trade_rate_per_min,
                "median_gap_sec": median_gap_sec,
                "mean_gap_sec": mean_gap_sec,
                "gap_cv": gap_cv,
                "burst_frac": burst_frac,
                "n_assets": n_assets,
                "top_asset_share": top_asset_share,
                "asset_switch_rate": asset_switch_rate,
                "turnover": turnover,
                "pnl_sum": pnl_sum,
                "pnl_mean": pnl_mean,
                "pnl_std": pnl_std,
                "win_rate": win_rate,
                "payoff_ratio": payoff_ratio,
                "pnl_skew_proxy": pnl_skew_proxy,
            }
        )

    return pd.DataFrame(rows)


core_windows = compute_core_window_vector(df, window_minutes=15, stride_minutes=5)
core_windows.head()


,window_start,window_end,n_trades,trade_rate_per_min,median_gap_sec,mean_gap_sec,gap_cv,burst_frac,n_assets,top_asset_share,asset_switch_rate,turnover,pnl_sum,pnl_mean,pnl_std,win_rate,payoff_ratio,pnl_skew_proxy
0,2025-01-01 13:34:00+00:00,2025-01-01 13:49:00+00:00,2,0.133333,220.0,220.0,0.0,0.0,1,1.0,0.0,0.11795,-20.65,-10.325,4.617407,0.0,NaN,-2.0
1,2025-01-01 13:39:00+00:00,2025-01-01 13:54:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
2,2025-01-01 13:44:00+00:00,2025-01-01 13:59:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
3,2025-01-01 13:49:00+00:00,2025-01-01 14:04:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
4,2025-01-01 13:54:00+00:00,2025-01-01 14:09:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN


## Training dataframe (one row per window)

Each window is treated as a session, so the training dataframe is simply the
core window features with a `session_id` per window.


In [45]:
def add_overtrading_indicators_per_window(
    core_windows: pd.DataFrame,
    window_minutes: int = 15,
) -> pd.DataFrame:
    """Add overtrading indicators per window (session)."""
    if core_windows.empty:
        return pd.DataFrame()

    dfw = core_windows.copy().sort_values("window_start").reset_index(drop=True)

    # Per-window turnover per hour
    if "turnover" in dfw.columns:
        dfw["turnover_per_hour"] = dfw["turnover"] / (window_minutes / 60.0)
    else:
        dfw["turnover_per_hour"] = float("nan")

    return dfw


def build_training_df(
    files: list[Path],
    window_minutes: int = 15,
    stride_minutes: int = 5,
) -> pd.DataFrame:
    all_rows = []
    for path in files:
        df_local = load_trades(path)
        core = compute_core_window_vector(
            df_local, window_minutes=window_minutes, stride_minutes=stride_minutes
        )
        if core.empty:
            continue

        core["session_id"] = core["window_start"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")
        core["source_file"] = path.name
        core["is_calm"] = 1 if ("calm" in path.name or "balanced" in path.name) else 0

        core = add_overtrading_indicators_per_window(core, window_minutes=window_minutes)
        all_rows.append(core)

    return pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame()


training_df = build_training_df(files, window_minutes=15, stride_minutes=5)
training_df.head()


,window_start,window_end,n_trades,trade_rate_per_min,median_gap_sec,mean_gap_sec,gap_cv,burst_frac,n_assets,top_asset_share,...,pnl_sum,pnl_mean,pnl_std,win_rate,payoff_ratio,pnl_skew_proxy,session_id,source_file,is_calm,turnover_per_hour
0,2025-01-01 13:34:00+00:00,2025-01-01 13:49:00+00:00,2,0.133333,220.0,220.0,0.0,0.0,1,1.0,...,-20.65,-10.325,4.617407,0.0,NaN,-2.0,2025-01-01T13:34:00Z,balanced_example.csv,1,0.471801
1,2025-01-01 13:39:00+00:00,2025-01-01 13:54:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,...,0.00,NaN,NaN,NaN,NaN,NaN,2025-01-01T13:39:00Z,balanced_example.csv,1,NaN
2,2025-01-01 13:44:00+00:00,2025-01-01 13:59:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,...,0.00,NaN,NaN,NaN,NaN,NaN,2025-01-01T13:44:00Z,balanced_example.csv,1,NaN
3,2025-01-01 13:49:00+00:00,2025-01-01 14:04:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,...,0.00,NaN,NaN,NaN,NaN,NaN,2025-01-01T13:49:00Z,balanced_example.csv,1,NaN
4,2025-01-01 13:54:00+00:00,2025-01-01 14:09:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,...,0.00,NaN,NaN,NaN,NaN,NaN,2025-01-01T13:54:00Z,balanced_example.csv,1,NaN


In [46]:
training_df.head()


,window_start,window_end,n_trades,trade_rate_per_min,median_gap_sec,mean_gap_sec,gap_cv,burst_frac,n_assets,top_asset_share,...,pnl_sum,pnl_mean,pnl_std,win_rate,payoff_ratio,pnl_skew_proxy,session_id,source_file,is_calm,turnover_per_hour
0,2025-01-01 13:34:00+00:00,2025-01-01 13:49:00+00:00,2,0.133333,220.0,220.0,0.0,0.0,1,1.0,...,-20.65,-10.325,4.617407,0.0,NaN,-2.0,2025-01-01T13:34:00Z,balanced_example.csv,1,0.471801
1,2025-01-01 13:39:00+00:00,2025-01-01 13:54:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,...,0.00,NaN,NaN,NaN,NaN,NaN,2025-01-01T13:39:00Z,balanced_example.csv,1,NaN
2,2025-01-01 13:44:00+00:00,2025-01-01 13:59:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,...,0.00,NaN,NaN,NaN,NaN,NaN,2025-01-01T13:44:00Z,balanced_example.csv,1,NaN
3,2025-01-01 13:49:00+00:00,2025-01-01 14:04:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,...,0.00,NaN,NaN,NaN,NaN,NaN,2025-01-01T13:49:00Z,balanced_example.csv,1,NaN
4,2025-01-01 13:54:00+00:00,2025-01-01 14:09:00+00:00,0,0.000000,NaN,NaN,NaN,NaN,0,NaN,...,0.00,NaN,NaN,NaN,NaN,NaN,2025-01-01T13:54:00Z,balanced_example.csv,1,NaN


In [47]:
# Save training data to CSV in this folder (absolute path)
output_path = Path("/Users/locsforstudygmail.com/Desktop/qhacks 2026/CogniTrade/backend/models/overtrading_model/data_analysis/data_preprocessing_training.csv")
if not training_df.empty:
    training_df.to_csv(output_path, index=False)
output_path


PosixPath('/Users/locsforstudygmail.com/Desktop/qhacks 2026/CogniTrade/backend/models/overtrading_model/data_analysis/data_preprocessing_training.csv')

In [48]:
# Duplicate cell removed — single definitions live above.


In [49]:
# Duplicate cell removed — save CSV lives above.
